# 理解 keras api


## layers 层 神经网络的基础

第二章已经接触过层的概念了,layer 是神经网络的基础结构.一个 layer 是一个数据处理模块,接收一个或多个张量输入,输出一个或多个张量.一些 layer 是无状态的,但大部分 layer 是有状态的(layer 的权重),layer 的权重是一个或几个由随机梯度下降得来的张量,储存有训练的来的知识.

不同类型的 layer 适用于处理不同的数据

- 简单矢量数据 -> 2D 张量(samples, features) -> 全连接层(密集连接层,Dense class in Keras)
- 时间序列化数据 -> 3D 张量(samples, timesteps, features) -> LSTM or Conv1D(一维卷积层)
- 图片数据 -> 4D 张量(samples, height, width, features) -> Conv2D(二维卷积层)

layer 可以被当作乐高积木,一层一层搭建起我们需要的神经网络.


keras 中 layer 的基础是 Layer Class

- keras 中所有代码都是以 layer 类为基础,或者是与 layer 类互动的对象.
- keras 类封装了权重和一些计算过程.例如权重的定义 build (也能在 init 中创建),权重的计算 call


In [1]:
from tensorflow import keras


class SimpleDense(keras.layers.Layer):  #所有层都是Layer的子类
    def __init__(self, units, activation=None):
        super().__init__()
        self.units = units
        self.activation = activation

    def build(self, input_shape):  #权重在build中初始化
        input_dim = input_shape[-1]
        self.W = self.add_weight(
            shape=(input_dim, self.units),
            initializer="random_normal")  #add_weight 是创建权重的快捷方法
        self.b = self.add_weight(shape=(self.units, ), initializer="zeros")

    def call(self, inputs):  #call 中定义了前向传播
        y = tf.matmul(inputs, self.W) + self.b
        if self.activation is not None:
            y = self.activation(y)
        return y

在 3.5 我们使用 tensorflow 实现了一个简单的线性分类器,上面是与之相同的 dense 层定义.


In [2]:
import tensorflow as tf

my_dense = SimpleDense(units=32,activation=tf.nn.relu)
input_tensor = tf.ones(shape=(2,784))
output_tensor = my_dense(input_tensor)
print(output_tensor)

tf.Tensor(
[[0.         1.8157713  0.         0.         0.         2.7955594
  0.         1.947442   0.91128755 2.192247   0.         0.00820488
  0.22186206 0.         0.36831257 2.4243002  0.7376876  0.
  0.         0.         0.         1.2063926  0.         0.33281553
  0.         1.5650915  0.         0.1085524  0.         0.
  1.9263895  1.5476991 ]
 [0.         1.8157713  0.         0.         0.         2.7955594
  0.         1.947442   0.91128755 2.192247   0.         0.00820488
  0.22186206 0.         0.36831257 2.4243002  0.7376876  0.
  0.         0.         0.         1.2063926  0.         0.33281553
  0.         1.5650915  0.         0.1085524  0.         0.
  1.9263895  1.5476991 ]], shape=(2, 32), dtype=float32)


定义了 SimpleDense,keras 可以像函数一样使用这些对象.

- 既然不用直接调用 call 和 build 为何还有定义这俩方法?
- 回答是希望能及时创建状态??


## 运行时,自动输入/输出规模判别

堆乐高积木时,虽然很自由,但也只能将能拼在一块的积木拼在一起.前面提到的每个例子,神经网络的每一层输入输出的张量都有明确的 shape.


In [22]:
from tensorflow.keras import layers
layer = layers.Dense(32,activation='relu')

这一层明确只返回一个张量,同时第一个轴有 32 个维度.它的下一层只能是接收 1D 32 维张量输入的 layer.


In [24]:
from tensorflow.keras import models
from tensorflow.keras import layers
model = models.Sequential([
    layers.Dense(32, activation="relu"),
    layers.Dense(32)
])

使用 keras 时,大部分情况下不需要担心 layer 之间输入输出规模匹配的问题,定义时会自动推断.


第二章实现的 Dense layer 中我们要求创建 layer 时,显式的传入层的输入大小.实际上这是一个失败的做法,这等于创建一个 layer 时,还需要明确知道上一层的输出规模.

如果决定上一层输出规模的规则非常复杂,那创建这一层 layer 会异常的艰难.


In [ ]:
def __call__(self, inputs):
    if not self.built:
         self.build(inputs.shape)
         self.built = True
    return self.call(inputs)

实际上上文提到的 build 方法就是一个解决之道.输入规模影响的是这一层 layer 权重的创建,在不知道输入规模情况下实例 layer 那只能不在实例化时直接创建权重了.

将权重创建移到 build 方法中,在第一次被调用时执行 build 方法(通过 call 方法),build 接收该层所见的第一个输入规模创建权重.


In [26]:
model = keras.Sequential([
    SimpleDense(32, activation="relu"),
    SimpleDense(64, activation="relu"),
    SimpleDense(32, activation="relu"),
    SimpleDense(10, activation="softmax")
])

自动的输入/输出规模推理,简化后就是上面的 keras layer 的定义了.

要注意: layer 的 call 方法,不止调用 build,还有完成了其他很多过程.

只需要记住,定义自己的 layer 时,将前向传播(计算输出)的过程放在 call 方法中.


## 从 layer 到 model

一个深度学习模型算是许多 layer 组成的图.在 keras 中对应是 model 类.

前面的例子中,我们见到的是顺序模型,只是简单的 layer 的层叠,一个层的输出是另一个层的输入.但是随着学习深入,还有许多其他的网络拓扑结构.

- 双分支网络
- 多头网络
- 剩余连接网络


![transformer0001.png](./transformer0001.png)

这是一个处理文件数据常见的网络拓扑,我们会在第 11 章见到.

定义复杂 model 的方法主要有两种,我们会在 7 章学习.

- 直接使用 model 类完成
- 使用 Functional API


第一章我们提到过,机器学习可以描述为利用反馈信号,在预定的可能空间内寻找对输入数据的有效表示.model 的网络拓扑限定了可能搜索的空间.为了从数据中学习,必须首先假设数据是什么类型,然后选择正确的网络架构.

如何选择正确的网络架构是一门艺术,不是科学.尽管有一些最佳实践和原则可以依靠,但是只有实践才能帮你完成正确的选择.这是接下来几章的重要内容.


## 编译 && 训练模型

一旦选定了 model,还有 3 个步骤需要完成:

- 损失函数: 在训练过程中被最小化的数值.衡量训练是否成功的标准.
- 优化器: 决定如果根据损失函数,更新网络权重.一般是随机梯度下降(SGD)或其一个变体.
- 指标: 用来在监视的指标,例如对分类问题就是分类的准确性.指标不会影响训练过程.训练也不会根据指标优化.

一旦确定了上述 3 样,就可以开始训练模型了.训练模型是使用内置的 compile 或 fit 方法完成.另外也可以定义自己的训练方法(第6章)


In [33]:
model = keras.Sequential([keras.layers.Dense(1)])
model.compile(optimizer="rmsprop",
              loss="mean_squared_error",
              metrics=["accuracy"])

上面是 model.compile 的一个示例.这里损失函数 优化器 和指标都是以字符串的形式传入.实际上这些字符串会展开成对应的 keras 类实例.例如 `"rmsprop"` 会被展开成 `keras.optimizers.RMSprop()`.


In [29]:
model.compile(optimizer=keras.optimizers.RMSprop(),
              loss=keras.losses.MeanSquaredError(),
              metrics=[keras.metrics.BinaryAccuracy()])

# model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=1e-4),
#              loss=my_custom_loss,
#              metrics=[my_custom_metric_1, my_custom_metric_2])

传入字符串是为了简化,直接传入实例对象也是可以的.

直接传入对象在需要定制化时更方便.


第 6 章的时候,我们会学习到如何创建自定义的损失函数和指标.一般而言,我们无需从头开始创建自定义的损失函数 优化器和指标.你需要的可能已经在 keras 内置了.

优化器

- `SGD()`: 随机梯度下降,每次选择小批量样本而不是全部.解决随机小批量样本问题,还有自适应学习率和容易陷入局部最优化问题.
- `Adagrad()`: 自适应梯度下降,迭代次数和累积梯度,对学习率进行自动衰减.没有考虑迭代衰减,极端情况，开始的梯度特别大，而后的比较小,则学习率基本不会变化了,也就算不算自适应学习率了.
- `RMSprop()`: 类似于 Adagrad,加入迭代衰减.
- `Adam()`: 算是混合 SGD 和 RMSprop,~~迄今为止最好的优化器.~~(谁都不排除有更好的)
- etc

损失函数

- `CategoricalCrossentropy()`
- `SparseCategoricalCrossentropy()`
- `BinaryCrossentropy()`
- `MeanSquaredError()`
- `KLDivergence()`
- `CosineSimilarity()`
- `Etc.

指标

- `CategoricalAccuracy()`
- `SparseCategoricalAccuracy()`
- `BinaryAccuracy()`
- `AUC()`
- `Precision()`
- `Recall()`
- `Etc.


## 选择损失函数

模型训练的过程可以看作是将损失函数最小化的过程,损失函数选择决定了模型最终的结果.

对于一些常见问题,有一些选择损失函数的准则.例如二分类问题选择`CategoricalCrossentropy` 多分类问题选择 `SparseCategoricalAccuracy`.只有当处理新的研究问题时,才会需要自定义损失函数.接下来几章会详述常见问题应该选择那些损失函数.


## fit 方法(训练 model)

fit 方法,训练 model

- data: 训练数据,通常是 numpy 数组,或者 tensorflow 的 dataset.
- 训练的轮次,fit 应该在 data 上迭代多少次.
- mini-batch 选择的批次大小,在权重更新计算梯度时,考虑的训练样本数量.


In [34]:
# 前置条件
import numpy as np

num_samples_per_class = 1000
negative_samples = np.random.multivariate_normal(
    mean=[0, 3], cov=[[1, 0.5],[0.5, 1]], size=num_samples_per_class)
positive_samples = np.random.multivariate_normal(
    mean=[3, 0], cov=[[1, 0.5],[0.5, 1]], size=num_samples_per_class)

inputs = np.vstack((negative_samples, positive_samples)).astype(np.float32)

targets = np.vstack((np.zeros((num_samples_per_class, 1), dtype="float32"),
                     np.ones((num_samples_per_class, 1), dtype="float32")))

model = keras.Sequential([keras.layers.Dense(1)])
model.compile(optimizer="sgd",
              loss="mean_squared_error",
              metrics=["accuracy"])

history = model.fit(
    inputs,
    targets,
    epochs=5,
    batch_size=128
)

Epoch 1/5
16/16 [==============================] - 1s 1ms/step - loss: 3.3322 - accuracy: 0.5935
Epoch 2/5
16/16 [==============================] - 0s 2ms/step - loss: 0.0547 - accuracy: 0.9765
Epoch 3/5
16/16 [==============================] - 0s 3ms/step - loss: 0.0242 - accuracy: 0.9980
Epoch 4/5
16/16 [==============================] - 0s 2ms/step - loss: 0.0239 - accuracy: 0.9980
Epoch 5/5
16/16 [==============================] - 0s 2ms/step - loss: 0.0239 - accuracy: 0.9980


In [35]:
history.history

{'loss': [3.332199811935425,
  0.05465351790189743,
  0.02423948235809803,
  0.023922046646475792,
  0.023885667324066162],
 'accuracy': [0.593500018119812,
  0.9764999747276306,
  0.9980000257492065,
  0.9980000257492065,
  0.9980000257492065]}

fit 方法会返回一个 History 对象,可以用来记录训练过程中的详细信息.


## 验证模型

模型最终不是工作在训练集上的,最终我们的目的是获得总体表现良好的模型,特别是在训练集以外的数据上.过度训练->过拟合.过少训练 ->欠拟合.

通常的做法是从训练集中保留一部分作为验证集,验证机不会用来训练,只用来评估模型.

In [42]:
model = keras.Sequential([keras.layers.Dense(1)])
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=0.1),
              loss=keras.losses.MeanSquaredError(),
              metrics=[keras.metrics.BinaryAccuracy()])

indices_permutation = np.random.permutation(len(inputs))
shuffled_inputs = inputs[indices_permutation]
shuffled_targets = targets[indices_permutation]

num_validation_samples = int(0.3 * len(inputs))
val_inputs = shuffled_inputs[:num_validation_samples]
val_targets = shuffled_targets[:num_validation_samples]
training_inputs = shuffled_inputs[num_validation_samples:]
training_targets = shuffled_targets[num_validation_samples:]

model.fit(
    training_inputs,
    training_targets,
    epochs=5,
    batch_size=16,
    validation_data=(val_inputs, val_targets)
)

Epoch 1/5
88/88 [==============================] - 1s 7ms/step - loss: 0.2656 - binary_accuracy: 0.9157 - val_loss: 0.0305 - val_binary_accuracy: 0.9950
Epoch 2/5
88/88 [==============================] - 0s 3ms/step - loss: 0.0679 - binary_accuracy: 0.9600 - val_loss: 0.0586 - val_binary_accuracy: 0.9850
Epoch 3/5
88/88 [==============================] - 0s 2ms/step - loss: 0.0706 - binary_accuracy: 0.9571 - val_loss: 0.0291 - val_binary_accuracy: 0.9967
Epoch 4/5
88/88 [==============================] - 0s 2ms/step - loss: 0.0687 - binary_accuracy: 0.9564 - val_loss: 0.1036 - val_binary_accuracy: 0.9417
Epoch 5/5
88/88 [==============================] - 0s 3ms/step - loss: 0.0756 - binary_accuracy: 0.9557 - val_loss: 0.0316 - val_binary_accuracy: 0.9933


fit 方法中使用 validation_data 传入验证数据.可以是 NumPy 数组或 TensorFlow dataset.

传入 validation_data 时, 会在每个 epoch 结束时调用一次验证数据.验证集的损失和指标严格与测试集分离,val_loss and val_binary_accuracy.


In [43]:
loss_and_metrics = model.evaluate(val_inputs, val_targets, batch_size=128)
print(loss_and_metrics)

5/5 [==============================] - 0s 2ms/step - loss: 0.0316 - binary_accuracy: 0.9933
[0.03157958388328552, 0.9933333396911621]


如果是训练完成后,再查看模型在验证集的数据,可以调用 model 的 evaluate 方法.


## 模型应用

模型训练完成后,开始进入应用阶段.


In [ ]:
# predictions = model(new_inputs)

获取模型预测结果的办法我们已经见过了.直接调用 model 传入数据即可.


In [ ]:
# predictions = model.predict(new_inputs, batch_size=128)

直接调用是一次性获取全部结果.如果数据集非常大,这样直接调用失败.更好的做法是调用 model.predict 方法,分批获取结果.

predict 方法还有个好处是可以处理 TensorFlow Dataset
